<a href="https://colab.research.google.com/github/Salsabil-Ouni/Cv-Generative-Chatbot-/blob/main/Langchain_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_smimport
!python -m spacy download en_core_web_lg # Large SpaCy model for English language
import numpy as np
import re # regular expressions
import spacy # NLU library

from collections import defaultdict
from sklearn.svm import SVC # Support Vector Classification model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-19 11:43:06.322093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 11:43:07.512552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

✘ No compatible package found for 'en_core_web_smimport' (spaCy
v3.5.3)

2023-06-19 11:43:14.981736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 11:43:16.134882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc

In [ ]:
!pip install <model_s3_url> # tar.gz file e.g. from release notes like https://github.com/explosion/spacy-models/releases//tag/en_core_web_lg-2.3.1
!pip install en_core_web_lg
import spacy

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `pip install <model_s3_url> # tar.gz file e.g. from release notes like https://github.com/explosion/spacy-models/releases//tag/en_core_web_lg-2.3.1'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
output_format = "IN: {input}\nOUT: {output}\n" + "_"*50


In [ ]:
responses_exact = {
    "what would you like to eat tonight?": "Pasta with salmon and red pesto please!",
    "what time will you be home tonight?": "I will be home around 6 pm.",
    "default": "I love you too!"
}

def respond_exact(text):
    response = responses_exact.get(text.lower(), responses_exact['default'])
    return(output_format.format(input=text, output=response))

In [ ]:
print(respond_exact("What would you like to eat tonight?"))
print("_"*50)
print(respond_exact("What time will you be home tonight?"))
print("_"*50)
print(respond_exact("I just found out my boss is leaving the company."))

IN: What would you like to eat tonight?
OUT: Pasta with salmon and red pesto please!
__________________________________________________
__________________________________________________
IN: What time will you be home tonight?
OUT: I will be home around 6 pm.
__________________________________________________
__________________________________________________
IN: I just found out my boss is leaving the company.
OUT: I love you too!
__________________________________________________


In [ ]:
import re
intent_keywords = {
    'dinner_preference': ['eat', 'dinner', 'food', 'cook', 'craving'],
    'arrival_time': ['time', 'when', 'get here', 'be home']
}
# Create a dictionary of patterns
patterns = {intent: re.compile('|'.join(keys)) for intent, keys in intent_keywords.items()}

# Define a function to find the intent of a message
def get_intent_re(message):
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message
        if pattern.search(message):
            return(intent)
        else:
            return('default')

responses_re = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "default":"I like you too!"
}

def respond_re(text):
    response = responses_re.get(get_intent_re(text))
    return(output_format.format(input=text, output=response))
print(respond_re("what would you like to eat tonight?"))
print(respond_re("what time will you be home tonight?"))
print(respond_re("I just food out my boss is leaving the company."))

IN: what would you like to eat tonight?
OUT: Pasta with salmon and red pesto please!
__________________________________________________
IN: what time will you be home tonight?
OUT: I like you too!
__________________________________________________
IN: I just food out my boss is leaving the company.
OUT: Pasta with salmon and red pesto please!
__________________________________________________


In [ ]:
training_sentences = [
    "What would you like to have for dinner?",
    "What do you want to eat tonight?",
    "I don't know what to cook tonight.",
    "Do you have any cravings?",
    "Can I get you something to eat?",
    "What time will you be home?",
    "How much longer will you be?",
    "When can we expect you to get here?",
    "What's taking you so long?",
    "At what hour will you be here?"

]
training_intents = [
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "arrival_time",
    "arrival_time",
    "arrival_time",
    "arrival_time",
     "arrival_time"
]

In [ ]:
nlp=spacy.load('en_core_web_lg')

In [ ]:
X_train = np.zeros((len(training_sentences),
              nlp('sentences').vocab.vectors_length))

for i, sentence in enumerate(training_sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X_train[i, :] = doc.vector

In [ ]:
# Create a support vector classifier
clf = SVC(C=1, gamma="auto", probability=True)

# Fit the classifier using the training data
clf.fit(X_train, training_intents)

SVC(C=1, gamma='auto', probability=True)

In [ ]:
def get_intent_ml(text):
    doc = nlp(text)
    return(clf.predict([doc.vector])[0])

In [ ]:
responses_ml = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "default":"I like you too!"
}

def respond_ml(text):
    response = responses_ml.get(get_intent_ml(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [ ]:
print(respond_ml("what would you like to eat tonight?"))
print(respond_ml("what time will you be home tonight?"))
print(respond_ml("l"))

IN: what would you like to eat tonight?
OUT: Pasta with salmon and red pesto please!
__________________________________________________
IN: what time will you be home tonight?
OUT: I will be home around 6 pm.
__________________________________________________
IN: l
OUT: I will be home around 6 pm.
__________________________________________________


In [ ]:
def get_intent_ml_2(text):
    """
        Returns the intent from a given text, unless the model is not sure, in which case 'default' is returned
    """
    doc = nlp(text)
    max_proba = max(clf.predict_proba([doc.vector])[0])
    if(max_proba == 0.5):
        return('default')
    else:
        return(clf.predict([doc.vector])[0])

def respond_ml_2(text):
    response = responses_ml.get(get_intent_ml_2(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [ ]:
print(respond_ml(  'flowers'))
print(respond_ml_2('flowers'))

IN: flowers
OUT: I will be home around 6 pm.
__________________________________________________
IN: flowers
OUT: I will be home around 6 pm.
__________________________________________________


In [ ]:
def get_all_entities(text):
    """
        Get all entities in a given text, in a text: label_ dictionary
    """
    doc = nlp(text)

    d = defaultdict(list)
    for ent in doc.ents:
        d[ent.label_].append(ent.text)
    return(d)

In [ ]:
test_ents = get_all_entities('what would you like to eat tonight?, or next tuesday or wednesday fish football Bengals')
print(sorted(test_ents.items()))

[('DATE', ['next tuesday', 'wednesday']), ('TIME', ['tonight'])]


In [ ]:
policy = {
    ("dinner_preference", "time and date"): "I want to eat pasta",
    ("dinner_preference", "time only"): "I want to eat pasta",
    ("dinner_preference", "date only"): "I want to eat pasta",
    ("dinner_preference", "none"): "When?",
    ("arrival_time", "time and date"): "I will be home at six",
    ("arrival_time", "time only"): "I will be home at six",
    ("arrival_time", "date only"): "I will be home at six",
    ("arrival_time", "none"): "When?",
    ("default", "none"): "What do you want?",
}

In [ ]:
def respond_ml_3(text):
    """Check for specification of date and time
        If not specified, ask for clarification
    """
    intent = get_intent_ml_2(text)

    if intent != 'default':
        entities = get_all_entities(text)
        if 'TIME' in entities and 'DATE' in entities:
            specification = 'time and date'
            time = ' and '.join(entities['DATE']) + ' at ' + ' and '.join(entities['TIME'])
        elif 'TIME' in entities:
            specification = 'time only'
            time = ' and '.join(entities['TIME'])
        elif 'DATE' in entities:
            specification = 'date only'
            time = ' and '.join(entities['DATE'])
        else:
            specification = 'none'
            time = ""
    else:
        specification = 'none'
        time = ""

    response = policy.get((intent, specification)) + ' ' + time
    return(output_format.format(input=text, output=response))